In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the dataset
data = pd.read_csv('data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Display the first few rows of the dataset
data.head()
